# 1D Convolution on CPU

## 1. Set-up 

In [1]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/drive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/drive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/drive/MyDrive/ece5545
%cd /content/drive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/drive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/drive/MyDrive/ece5545

PROJECT_ROOT = f"/content/drive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/drive/MyDrive/ece5545’: File exists
/content/drive/MyDrive/ece5545
fatal: destination path 'a3-NamanMakkar' already exists and is not an empty directory.
/content/drive/MyDrive/ece5545/a3-NamanMakkar
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/drive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

Verify the following cell prints your github repository.

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2. Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 MB 3.8 MB/s eta 0:00:00


## 3. Implement `make_conv1d_cpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_cpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM schedule and the TVM operator for 
1. Input $x$
2. Input $y$
3. Output $out$

The schedule should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells the usage.

In [10]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_conv1d_cpu_scheduler
import os
import tvm
from tvm import te

M = 4096
N = 128
dtype = 'float32'
a_np = np.random.rand(M).astype(dtype)
w_np = np.random.rand(N).astype(dtype)
b_np = np.convolve(a_np, w_np)

s, A, W, B = make_conv1d_cpu_scheduler(M, N)
func = tvm.build(s, [A, W, B], "llvm")

dev = tvm.cpu()
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M+N-1), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1D conv TVM runtime: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [0.5634176  0.5735868  1.1364146  ... 0.9448417  0.51518357 0.47124672]
Output: [0.5634176  0.5735868  1.1364148  ... 0.9448417  0.51518357 0.47124672]
1D conv TVM runtime: 0.293483 ms


In [11]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((4096,), "float32"), W: T.Buffer((128,), "float32"), B: T.Buffer((4223,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "global_symbol": "main", "tir.noalias": T.bool(True)})
        padded_A = T.allocate([4350], "float32", "global")
        padded_A_local = T.allocate([31], "float32", "local")
        W_local = T.allocate([16], "float32", "local")
        padded_A_1 = T.Buffer((4350,), data=padded_A)
        for n in range(4350):
            A_1 = T.Buffer((4096,), data=A.data)
            padded_A_1[n] = T.if_then_else(n < 127 or 4223 <= n, T.float32(0), A_1[n - 127])
        for n_outer in range(264):
            B_1 = T.Buffer((4223,), data=B.data)
            for n_inner_init_s in range(16):
                if T.likely(n_outer * 16 + n_inner_init_s < 4223):
                    B_1[n_outer * 16 + n_inner_init_s] = T.fl

In [12]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_1dconv_cpu.py

/content/drive/MyDrive/ece5545/a3-NamanMakkar
============================= test session starts ==============================
platform linux -- Python 3.9.16, pytest-7.2.2, pluggy-1.0.0
rootdir: /content/drive/MyDrive/ece5545/a3-NamanMakkar
plugins: anyio-3.6.2
collected 15 items                                                             

tests/test_1dconv_cpu.py ...............                                 [100%]

============================= 15 passed in 18.90s ==============================
